# CARLE's Game Collaboration Notebook

## Agent/Policy Description

## Voting Instructions

Voting instructions will be added here in a later update

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import torch
import time

import pandas as pd
from carle.env import CARLE
from carle.mcl import RND2D, AE2D, SpeedDetector, PufferDetector
from collaboration.agents import ConvGRNNAgent, CMAPopulation

import bokeh
import bokeh.io as bio
from bokeh.io import output_notebook, show
from bokeh.plotting import figure

from bokeh.layouts import column, row
from bokeh.models import TextInput, Button, Paragraph
from bokeh.models import ColumnDataSource


from bokeh.events import DoubleTap, Tap

output_notebook()

In [ ]:
# initialize environment and add exploratin wrappers

max_steps = 512

env_0 = CARLE(use_cuda=True) 
env_1 = CARLE(use_cuda=True) 
env_2 = CARLE(use_cuda=True) 
env_3 = CARLE(use_cuda=True) 


In [ ]:

my_device = torch.device("cuda")
def modify_doc(doc):
        
    with torch.no_grad():
        #env = CARLE(use_cuda=True)

        #dot life rules
        env_0.birth = [3]
        env_0.survive = [2,3]
        env_1.birth = [3]
        env_1.survive = [2,3]
        env_2.birth = [3]
        env_2.survive = [2,3]
        env_3.birth = [3]
        env_3.survive = [2,3]

        global agent
        agent = CMAPopulation(ConvGRNNAgent, device="cuda")
        
        global obs
        obs_0 = env_0.reset()
        obs_1 = env_1.reset()
        obs_2 = env_1.reset()
        obs_3 = env_1.reset()

        p0 = figure(plot_width=3*256//2, plot_height=3*256//2)
        p1 = figure(plot_width=3*256//2, plot_height=3*256//2)
        p2 = figure(plot_width=3*256//2, plot_height=3*256//2)
        p3 = figure(plot_width=3*256//2, plot_height=3*256//2)

        global my_period
        my_period = 512

        # add a circle renderer with x and y coordinates, size, color, and alpha

        source_0 = ColumnDataSource(data=dict(my_image_0=[obs_0.squeeze().cpu().numpy()]))
        source_1 = ColumnDataSource(data=dict(my_image_1=[obs_1.squeeze().cpu().numpy()]))
        source_2 = ColumnDataSource(data=dict(my_image_2=[obs_2.squeeze().cpu().numpy()]))
        source_3 = ColumnDataSource(data=dict(my_image_3=[obs_3.squeeze().cpu().numpy()]))
                                            
        img_0 = p0.image(image='my_image_0',x=0, y=0, dw=256, dh=256, palette="Magma256", source=source_0)
        img_1 = p1.image(image='my_image_1',x=0, y=0, dw=256, dh=256, palette="Magma256", source=source_1)
        img_2 = p2.image(image='my_image_2',x=0, y=0, dw=256, dh=256, palette="Magma256", source=source_2)
        img_3 = p3.image(image='my_image_3',x=0, y=0, dw=256, dh=256, palette="Magma256", source=source_3)

        button_go = Button(sizing_mode="stretch_width", label="Run >")     
        button_slower = Button(sizing_mode="stretch_width",label="<< Slower")
        button_faster = Button(sizing_mode="stretch_width",label="Faster >>")

        input_birth = TextInput(value="B3")
        input_survive = TextInput(value="S23")
        
        button_birth = Button(sizing_mode="stretch_width", label="Update Birth Rules")
        button_survive = Button(sizing_mode="stretch_width", label="Update Survive Rules")
        
        button_pick_0 = Button(sizing_mode="stretch_width", label="Pick agent 0")
        button_pick_1 = Button(sizing_mode="stretch_width", label="Pick agent 1")
        button_pick_2 = Button(sizing_mode="stretch_width", label="Pick agent 2")
        button_pick_3 = Button(sizing_mode="stretch_width", label="Pick agent 3")
        
        
        message = Paragraph()
        

        def update():
            global obs_0
            global obs_1
            global obs_2
            global obs_3
            
            global stretch_pixel
            
            global action_0
            global action_1
            global action_2
            global action_3
            
            global total_steps
            
            obs_0, r, d, i = env_0.step(action_0)
            obs_1, r, d, i = env_1.step(action_1)
            obs_2, r, d, i = env_2.step(action_2)
            obs_3, r, d, i = env_3.step(action_3)

            action_0 = agent(obs_0, agent_index=0)
            action_1 = agent(obs_1, agent_index=1)
            action_2 = agent(obs_2, agent_index=2)
            action_3 = agent(obs_3, agent_index=3)
                        
            padded_action_0 = stretch_pixel/2 + env_0.action_padding(action_0).squeeze()
            padded_action_1 = stretch_pixel/2 + env_1.action_padding(action_1).squeeze()
            padded_action_2 = stretch_pixel/2 + env_2.action_padding(action_2).squeeze()
            padded_action_3 = stretch_pixel/2 + env_3.action_padding(action_3).squeeze()
            
            my_img_0 = (padded_action_0*2 + obs_0.squeeze()).cpu().numpy()
            my_img_1 = (padded_action_1*2 + obs_1.squeeze()).cpu().numpy()
            my_img_2 = (padded_action_2*2 + obs_2.squeeze()).cpu().numpy()
            my_img_3 = (padded_action_3*2 + obs_3.squeeze()).cpu().numpy()
            
            my_img_0[my_img_0 > 3.0] = 3.0
            my_img_1[my_img_1 > 3.0] = 3.0
            my_img_2[my_img_2 > 3.0] = 3.0
            my_img_3[my_img_3 > 3.0] = 3.0
            
            new_data_0 = dict(my_image_0=[my_img_0])
            new_data_1 = dict(my_image_1=[my_img_1])
            new_data_2 = dict(my_image_2=[my_img_2])
            new_data_3 = dict(my_image_3=[my_img_3])

            source_0.stream(new_data_0, rollover=1)
            source_1.stream(new_data_1, rollover=1)
            source_2.stream(new_data_2, rollover=1)
            source_3.stream(new_data_3, rollover=1)

        def go():

            if button_go.label == "Run >":
                my_callback = doc.add_periodic_callback(update, my_period)
                button_go.label = "Pause"
                #doc.remove_periodic_callback(my_callback)

            else:
                doc.remove_periodic_callback(doc.session_callbacks[0])
                button_go.label = "Run >"

        def faster():
            global my_period
            my_period = max([my_period / 2, 1])
            go()
            go()

        def slower():

            global my_period
            my_period = min([my_period * 2, 8192])
            go()
            go()
                             
        def reset():
            global obs_0
            global obs_1
            global obs_2
            global obs_3
            
            global stretch_pixel
            
            global action_0
            global action_1
            global action_2
            global action_3
            
            global total_steps

            obs_0 = env_0.reset()
            obs_1 = env_1.reset()
            obs_2 = env_2.reset()
            obs_3 = env_3.reset()

            action_0 = agent(obs_0, agent_index=0)
            action_1 = agent(obs_1, agent_index=1)
            action_2 = agent(obs_2, agent_index=2)
            action_3 = agent(obs_3, agent_index=3)
            
            stretch_pixel = torch.zeros_like(obs_0).squeeze()
            stretch_pixel[0,0] = 3
                        
            new_data_0 = dict(my_image_0=[(stretch_pixel + obs_0.squeeze()).cpu().numpy()])
            new_data_1 = dict(my_image_1=[(stretch_pixel + obs_0.squeeze()).cpu().numpy()])
            new_data_2 = dict(my_image_2=[(stretch_pixel + obs_0.squeeze()).cpu().numpy()])
            new_data_3 = dict(my_image_3=[(stretch_pixel + obs_0.squeeze()).cpu().numpy()])
            
            source_0.stream(new_data_0, rollover=8)
            source_1.stream(new_data_1, rollover=8)
            source_2.stream(new_data_2, rollover=8)
            source_3.stream(new_data_3, rollover=8)
                             

        def pick_agent_0():
            global obs_0
            global obs_1
            global obs_2
            global obs_3
            
            global stretch_pixel
            
            global action_0
            global action_1
            global action_2
            global action_3
            
            global total_steps

            agent.step(rewards=[1.,0.,0.,0.])
            
            obs_0 = env_0.reset()
            obs_1 = env_1.reset()
            obs_2 = env_2.reset()
            obs_3 = env_3.reset()
            
            action_0 = agent(obs_0, agent_index=0)
            action_1 = agent(obs_1, agent_index=1)
            action_2 = agent(obs_2, agent_index=2)
            action_3 = agent(obs_3, agent_index=3)
            
            stretch_pixel = torch.zeros_like(obs_0).squeeze()
            stretch_pixel[0,0] = 3
                        
            new_data_0 = dict(my_image_0=[(stretch_pixel + obs_0.squeeze()).cpu().numpy()])
            new_data_1 = dict(my_image_1=[(stretch_pixel + obs_0.squeeze()).cpu().numpy()])
            new_data_2 = dict(my_image_2=[(stretch_pixel + obs_0.squeeze()).cpu().numpy()])
            new_data_3 = dict(my_image_3=[(stretch_pixel + obs_0.squeeze()).cpu().numpy()])
            
            source_0.stream(new_data_0, rollover=8)
            source_1.stream(new_data_1, rollover=8)
            source_2.stream(new_data_2, rollover=8)
            source_3.stream(new_data_3, rollover=8)
                             

        def pick_agent_1():
            global obs_0
            global obs_1
            global obs_2
            global obs_3
            
            global stretch_pixel
            
            global action_0
            global action_1
            global action_2
            global action_3
            
            global total_steps

            agent.step(rewards=[0.,1.,0.,0.])
            
            obs_0 = env_0.reset()
            obs_1 = env_1.reset()
            obs_2 = env_2.reset()
            obs_3 = env_3.reset()

            action_0 = agent(obs_0, agent_index=0)
            action_1 = agent(obs_1, agent_index=1)
            action_2 = agent(obs_2, agent_index=2)
            action_3 = agent(obs_3, agent_index=3)
            
            stretch_pixel = torch.zeros_like(obs_0).squeeze()
            stretch_pixel[0,0] = 3
                        
            new_data_0 = dict(my_image_0=[(stretch_pixel + obs_0.squeeze()).cpu().numpy()])
            new_data_1 = dict(my_image_1=[(stretch_pixel + obs_0.squeeze()).cpu().numpy()])
            new_data_2 = dict(my_image_2=[(stretch_pixel + obs_0.squeeze()).cpu().numpy()])
            new_data_3 = dict(my_image_3=[(stretch_pixel + obs_0.squeeze()).cpu().numpy()])
            
            source_0.stream(new_data_0, rollover=8)
            source_1.stream(new_data_1, rollover=8)
            source_2.stream(new_data_2, rollover=8)
            source_3.stream(new_data_3, rollover=8)

        def pick_agent_2():
            global obs_0
            global obs_1
            global obs_2
            global obs_3
            
            global stretch_pixel
            
            global action_0
            global action_1
            global action_2
            global action_3
            
            global total_steps

            agent.step(rewards=[0.,0.,1.,0.])
            
            obs_0 = env_0.reset()
            obs_1 = env_1.reset()
            obs_2 = env_2.reset()
            obs_3 = env_3.reset()

            action_0 = agent(obs_0, agent_index=0)
            action_1 = agent(obs_1, agent_index=1)
            action_2 = agent(obs_2, agent_index=2)
            action_3 = agent(obs_3, agent_index=3)
            
            stretch_pixel = torch.zeros_like(obs_0).squeeze()
            stretch_pixel[0,0] = 3

                        
            new_data_0 = dict(my_image_0=[(stretch_pixel + obs_0.squeeze()).cpu().numpy()])
            new_data_1 = dict(my_image_1=[(stretch_pixel + obs_0.squeeze()).cpu().numpy()])
            new_data_2 = dict(my_image_2=[(stretch_pixel + obs_0.squeeze()).cpu().numpy()])
            new_data_3 = dict(my_image_3=[(stretch_pixel + obs_0.squeeze()).cpu().numpy()])

            source_0.stream(new_data_0, rollover=8)
            source_1.stream(new_data_1, rollover=8)
            source_2.stream(new_data_2, rollover=8)
            source_3.stream(new_data_3, rollover=8)
                             

        def pick_agent_3():
            global obs_0
            global obs_1
            global obs_2
            global obs_3
            
            global stretch_pixel
            
            global action_0
            global action_1
            global action_2
            global action_3
            
            global total_steps

            agent.step(rewards=[0.,0.,0.,1.])
            
            obs_0 = env_0.reset()
            obs_1 = env_1.reset()
            obs_2 = env_2.reset()
            obs_3 = env_3.reset()

            action_0 = agent(obs_0, agent_index=0)
            action_1 = agent(obs_1, agent_index=1)
            action_2 = agent(obs_2, agent_index=2)
            action_3 = agent(obs_3, agent_index=3)
            
            stretch_pixel = torch.zeros_like(obs_0).squeeze()
            stretch_pixel[0,0] = 3
                        
            new_data_0 = dict(my_image_0=[(stretch_pixel + obs_0.squeeze()).cpu().numpy()])
            new_data_1 = dict(my_image_1=[(stretch_pixel + obs_0.squeeze()).cpu().numpy()])
            new_data_2 = dict(my_image_2=[(stretch_pixel + obs_0.squeeze()).cpu().numpy()])
            new_data_3 = dict(my_image_3=[(stretch_pixel + obs_0.squeeze()).cpu().numpy()])
            
            source_0.stream(new_data_0, rollover=8)
            source_1.stream(new_data_1, rollover=8)
            source_2.stream(new_data_2, rollover=8)
            source_3.stream(new_data_3, rollover=8)

        def set_birth_rules():
            env_0.birth_rule_from_string(input_birth.value)
            env_1.birth_rule_from_string(input_birth.value)
            env_2.birth_rule_from_string(input_birth.value)
            env_3.birth_rule_from_string(input_birth.value)

            my_message = "Rules updated to B"

            for elem in env_0.birth:
                my_message += str(elem)
            my_message += "/S"    

            for elem in env_0.survive:
                my_message += str(elem)

            message.text = my_message

            #reset()

        def set_survive_rules():
            env_0.survive_rule_from_string(input_survive.value)
            env_1.survive_rule_from_string(input_survive.value)
            env_2.survive_rule_from_string(input_survive.value)
            env_3.survive_rule_from_string(input_survive.value)

            my_message = "Rules updated to B"

            for elem in env_0.birth:
                my_message += str(elem)
            my_message += "/S"    

            for elem in env_0.survive:
                my_message += str(elem)

            message.text = my_message

            #reset()

        agent_on = True
                             

        global action_0
        global action_1
        global action_2
        global action_3
        
        action_0 = torch.zeros(1, 1, env_0.action_height, env_0.action_width).to(my_device)
        action_1 = torch.zeros(1, 1, env_0.action_height, env_0.action_width).to(my_device)
        action_2 = torch.zeros(1, 1, env_0.action_height, env_0.action_width).to(my_device)
        action_3 = torch.zeros(1, 1, env_0.action_height, env_0.action_width).to(my_device)


        button_birth.on_click(set_birth_rules)
        button_survive.on_click(set_survive_rules)
        button_go.on_click(go)
        button_faster.on_click(faster)
        button_slower.on_click(slower)

        button_pick_0.on_click(pick_agent_0)
        button_pick_1.on_click(pick_agent_1)
        button_pick_2.on_click(pick_agent_2)
        button_pick_3.on_click(pick_agent_3)
                             
                             
        control_layout = row(button_slower, button_go, button_faster)
        rule_layout = row(input_birth, button_birth, input_survive, button_survive)
                             
        
                        
        column0 = column(p0, button_pick_0)
        column1 = column(p1, button_pick_1)
        column2 = column(p2, button_pick_2)
        column3 = column(p3, button_pick_3)
        
        display_layout_0 = row(column0, column1)
        display_layout_1 = row(column2, column3)

        message_layout = row(message)

        doc.add_root(display_layout_0)
        doc.add_root(display_layout_1)
        doc.add_root(control_layout)
        doc.add_root(rule_layout)
        doc.add_root(message_layout)

        reset()

show(modify_doc)    

In [ ]:
agent(torch.zeros(1,1,256,256).to(my_device), agent_index=0).sum()